# 20220518

开始第三版带有晶界模型的测试

首先生成一个210晶界模型


In [ ]:
from kmcpy.event_generator import *
from kmcpy.external.pymatgen_structure import Structure
from kmcpy.external.pymatgen_local_env import CutOffDictNN

    
def build_distance_matrix_from_getnninfo_output(cutoffdnn_output={},verbose=False):
    """build a distance matrix from the output of CutOffDictNN.get_nn_info

    nn_info looks like: 
    [{'site': PeriodicSite: Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0), 'weight': 3.7390091507903174, 'site_index': 39, 'wyckoff_sequence': 15, 'local_index': 123, 'label': 'Si1'}, {'site': PeriodicSite: Na+ (-1.2831, -2.6519, 9.2421) [-0.3063, -0.3333, 0.4167], 'image': (-1, -1, 0), 'weight': 3.4778161424304046, 'site_index': 23, 'wyckoff_sequence': 17, 'local_index': 35, 'label': 'Na2'}, {'site': ...]
    
    or say:
    
    nn_info is a list, the elements of list is dictionary, the keys of dictionary are: "site":pymatgen.site, "wyckoff_sequence": ....
    
    Use the site.distance function to build matrix
    

    Args:
        cutoffdnn_output (nn_info, optional): nninfo. Defaults to reference_neighbor_sequences.

    Returns:
        np.2darray: 2d distance matrix, in format of numpy.array. The Column and the Rows are following the input sequence.
    """

    distance_matrix=np.zeros(shape=(len(cutoffdnn_output),len(cutoffdnn_output)))
        

    for sitedictindex1 in range(0,len(cutoffdnn_output)):
        for sitedictindex2 in range(0,len(cutoffdnn_output)):
            """Reason for jimage=[0,0,0]
            
            site.distance is calculated by frac_coord1-frac_coord0 and get the cartesian distance. Note that for the two sites in neighbors,  the frac_coord itself already contains the information of jimage. For exaple:Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0),  see that the frac_coord of this Si4+ is not normalized to (0,1)!

            .
            """
            distance_matrix[sitedictindex1][sitedictindex2]=cutoffdnn_output[sitedictindex1]["site"].distance(cutoffdnn_output[sitedictindex2]["site"],jimage=[0,0,0])
        
        
    
    return distance_matrix

li3clo_210=Structure.from_file("CONTCAR")
li3clo_210.to("cif","210.cif")
li3clo_210.add_oxidation_state_by_guess()

#local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","O2-"):4.0})
local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","Li+"):3.0})
#local_env_finder = CutOffDictNN({("Li+","Cl-"):2.8})
np.set_printoptions(precision=2,suppress=True)  
site_to_mark1=[]
site_to_mark2=[]
site_to_mark3=[]
for i in range(0,len(li3clo_210)):
    if "Li+" not in li3clo_210[i].species:
        continue
    
    #print(local_env_finder.get_nn_info(li3clo_210,i))
    print(len(local_env_finder.get_nn_info(li3clo_210,i)))
    """
    
    for #local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","O2-"):4.0})
        if len(local_env_finder.get_nn_info(li3clo_210,i))==5:
        print(local_env_finder.get_nn_info(li3clo_210,i))
        li3clo_210.replace(i,{"Na+":1.0})

    if len(local_env_finder.get_nn_info(li3clo_210,i))==4:
        print(local_env_finder.get_nn_info(li3clo_210,i))
        li3clo_210.replace(i,{"K+":1.0})
    """
    
    if len(local_env_finder.get_nn_info(li3clo_210,i))!=12:
        print(local_env_finder.get_nn_info(li3clo_210,i))
        print(build_distance_matrix_from_getnninfo_output(local_env_finder.get_nn_info(li3clo_210,i)))
        site_contain_li=[neighbor["site"].species for neighbor in local_env_finder.get_nn_info(li3clo_210,i)]
        li_num=0
        for site in site_contain_li:
            if "Li+" in site:
                li_num+=1
        print("Li num",li_num)

        if len(local_env_finder.get_nn_info(li3clo_210,i))==11:
            site_to_mark1.append(i)
        elif len(local_env_finder.get_nn_info(li3clo_210,i))==8:
            site_contain_li=[neighbor["site"].species for neighbor in local_env_finder.get_nn_info(li3clo_210,i)]
            li_num=0
            for site in site_contain_li:
                if "Li+" in site:
                    li_num+=1
            if li_num==6:
                site_to_mark2.append(i)
            else:
                site_to_mark3.append(i)
        else:
            raise ValueError()

for i in site_to_mark1:
    li3clo_210.replace(i,{"Na+":1.0})
for i in site_to_mark2:
    li3clo_210.replace(i,{"K+":1.0})
for i in site_to_mark3:
    li3clo_210.replace(i,{"Rb+":1.0})
li3clo_210.to("cif","marked210.cif")
    
    #print(local_env_finder.get_all_nn_info(li3clo_210))



For 210 grain boundary, there are several types of Li environment

1. General Lithium (larger green atom): 8 Li+ and 4 Cl- and 2 O2-
2. Gold: 7 Li+ and 4 Cl- and 2O2-
3. Pink: 4 Li+ and 4 Cl- and 1O2-
4. Purple: 6 Li+ and 2 Cl-  and 2O2-

**One important thing is that the distance matrix are different for type 2,3,4**

Starting initialize a new function to generate different kinds of clusters


In [21]:
from kmcpy.event_generator import *
from kmcpy.external.pymatgen_structure import Structure
from kmcpy.external.pymatgen_local_env import CutOffDictNN

    
def build_distance_matrix_from_getnninfo_output(cutoffdnn_output={},verbose=False):
    """build a distance matrix from the output of CutOffDictNN.get_nn_info

    nn_info looks like: 
    [{'site': PeriodicSite: Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0), 'weight': 3.7390091507903174, 'site_index': 39, 'wyckoff_sequence': 15, 'local_index': 123, 'label': 'Si1'}, {'site': PeriodicSite: Na+ (-1.2831, -2.6519, 9.2421) [-0.3063, -0.3333, 0.4167], 'image': (-1, -1, 0), 'weight': 3.4778161424304046, 'site_index': 23, 'wyckoff_sequence': 17, 'local_index': 35, 'label': 'Na2'}, {'site': ...]
    
    or say:
    
    nn_info is a list, the elements of list is dictionary, the keys of dictionary are: "site":pymatgen.site, "wyckoff_sequence": ....
    
    Use the site.distance function to build matrix
    

    Args:
        cutoffdnn_output (nn_info, optional): nninfo. Defaults to reference_neighbor_sequences.

    Returns:
        np.2darray: 2d distance matrix, in format of numpy.array. The Column and the Rows are following the input sequence.
    """

    distance_matrix=np.zeros(shape=(len(cutoffdnn_output),len(cutoffdnn_output)))
        

    for sitedictindex1 in range(0,len(cutoffdnn_output)):
        for sitedictindex2 in range(0,len(cutoffdnn_output)):
            """Reason for jimage=[0,0,0]
            
            site.distance is calculated by frac_coord1-frac_coord0 and get the cartesian distance. Note that for the two sites in neighbors,  the frac_coord itself already contains the information of jimage. For exaple:Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0),  see that the frac_coord of this Si4+ is not normalized to (0,1)!

            .
            """
            distance_matrix[sitedictindex1][sitedictindex2]=cutoffdnn_output[sitedictindex1]["site"].distance(cutoffdnn_output[sitedictindex2]["site"],jimage=[0,0,0])
        
        
    
    return distance_matrix



li3clo_210=Structure.from_file("CONTCAR")
li3clo_210.add_oxidation_state_by_guess()
#local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","O2-"):4.0})
local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","Li+"):3.0})

clusters=[]



for i in range(0,len(li3clo_210)):
    if "Li+" in li3clo_210[i].species:
        nn=local_env_finder.get_nn_info(li3clo_210,i)
        nn_elements=[neighbor["site"].species.elements[0] for neighbor in nn]
        #print(local_env_finder.get_cn_dict(li3clo_210,i))
        
        if local_env_finder.get_cn_dict(li3clo_210,i) not in clusters:
            clusters.append(local_env_finder.get_cn_dict(li3clo_210,i))
print(clusters)

[{'Li+': 8, 'Cl-': 4}, {'Li+': 7, 'Cl-': 4}, {'Cl-': 4, 'Li+': 4}, {'Li+': 6, 'Cl-': 2}]


# A new attempt on rearranging neighbor

In [32]:
import numpy as np

import json
import logging
from kmcpy.external.pymatgen_structure import Structure
from kmcpy.external.pymatgen_local_env import CutOffDictNN
import itertools

def build_distance_matrix_from_getnninfo_output(cutoffdnn_output=[],verbose=False):
    """build a distance matrix from the output of CutOffDictNN.get_nn_info

    nn_info looks like: 
    [{'site': PeriodicSite: Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0), 'weight': 3.7390091507903174, 'site_index': 39, 'wyckoff_sequence': 15, 'local_index': 123, 'label': 'Si1'}, {'site': PeriodicSite: Na+ (-1.2831, -2.6519, 9.2421) [-0.3063, -0.3333, 0.4167], 'image': (-1, -1, 0), 'weight': 3.4778161424304046, 'site_index': 23, 'wyckoff_sequence': 17, 'local_index': 35, 'label': 'Na2'}, {'site': ...]
    
    or say:
    
    nn_info is a list, the elements of list is dictionary, the keys of dictionary are: "site":pymatgen.site, "wyckoff_sequence": ....
    
    Use the site.distance function to build matrix
    

    Args:
        cutoffdnn_output (nn_info, optional): nninfo. Defaults to reference_neighbor_sequences.

    Returns:
        np.2darray: 2d distance matrix, in format of numpy.array. The Column and the Rows are following the input sequence.
    """

    distance_matrix=np.zeros(shape=(len(cutoffdnn_output),len(cutoffdnn_output)))
        

    for sitedictindex1 in range(0,len(cutoffdnn_output)):
        for sitedictindex2 in range(0,len(cutoffdnn_output)):
            """Reason for jimage=[0,0,0]
            
            site.distance is calculated by frac_coord1-frac_coord0 and get the cartesian distance. Note that for the two sites in neighbors,  the frac_coord itself already contains the information of jimage. For exaple:Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0),  see that the frac_coord of this Si4+ is not normalized to (0,1)!

            .
            """
            distance_matrix[sitedictindex1][sitedictindex2]=cutoffdnn_output[sitedictindex1]["site"].distance(cutoffdnn_output[sitedictindex2]["site"],jimage=[0,0,0])
        
        
    
    return distance_matrix


nasicon=Structure.from_cif("EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif",primitive=True)

center_Na1=[0,1]
local_env_finder = CutOffDictNN({('Na+','Na+'):4,('Na+','Si4+'):4})


reference_neighbor_sequences=sorted(sorted(local_env_finder.get_nn_info(nasicon,center_Na1[0]),key=lambda x:x["wyckoff_sequence"]),key = lambda x:x["label"])     

reference_distance_matrix=build_distance_matrix_from_getnninfo_output(reference_neighbor_sequences)

print(reference_neighbor_sequences)
np.set_printoptions(precision=2,suppress=True)

reference_na_distance_matrix=build_distance_matrix_from_getnninfo_output(reference_neighbor_sequences[0:6])


possible_arrange_na=[]

for possible_na_sequence in itertools.permutations(reference_neighbor_sequences[0:6]):
    if np.allclose(build_distance_matrix_from_getnninfo_output(possible_na_sequence),reference_na_distance_matrix,rtol=0.01):
        possible_arrange_na.append(possible_na_sequence)
        
print(len(possible_arrange_na))

reference_si_distance_matrix=build_distance_matrix_from_getnninfo_output(reference_neighbor_sequences[6:12])


possible_arrange_si=[]

for possible_si_sequence in itertools.permutations(reference_neighbor_sequences[6:12]):
    if np.allclose(build_distance_matrix_from_getnninfo_output(possible_si_sequence),reference_si_distance_matrix,rtol=0.01):
        possible_arrange_si.append(possible_si_sequence)

print(len(possible_arrange_si))


total_iter=0

for possible_complete_sequence in itertools.product(possible_arrange_na,possible_arrange_si):
    re_sorted_neighbors_list=[]
    
    for neighbor in possible_complete_sequence:

        re_sorted_neighbors_list.extend(list(neighbor))     

    if np.allclose(build_distance_matrix_from_getnninfo_output(re_sorted_neighbors_list),reference_distance_matrix,rtol=0.01):
        print(possible_complete_sequence)
        total_iter+=1
    pass

print(total_iter)
        



[{'site': PeriodicSite: Na+ (1.2831, 2.6519, 12.9389) [0.8897, 0.6103, 0.2500], 'image': (0, 0, 0), 'weight': 3.4778161424304033, 'site_index': 2, 'wyckoff_sequence': 0, 'local_index': 18, 'label': 'Na2'}, {'site': PeriodicSite: Na+ (1.6550, -2.4371, 12.9389) [0.6103, 0.2500, 0.8897], 'image': (0, 0, 0), 'weight': 3.4778161424304033, 'site_index': 3, 'wyckoff_sequence': 1, 'local_index': 19, 'label': 'Na2'}, {'site': PeriodicSite: Na+ (-2.9381, -0.2148, 12.9389) [0.2500, 0.8897, 0.6103], 'image': (0, 0, 0), 'weight': 3.477816142430404, 'site_index': 4, 'wyckoff_sequence': 2, 'local_index': 20, 'label': 'Na2'}, {'site': PeriodicSite: Na+ (-1.6550, 2.4371, 9.2421) [0.3897, 0.7500, 0.1103], 'image': (0, 0, 0), 'weight': 3.4778161424304015, 'site_index': 5, 'wyckoff_sequence': 3, 'local_index': 21, 'label': 'Na2'}, {'site': PeriodicSite: Na+ (2.9381, 0.2148, 9.2421) [0.7500, 0.1103, 0.3897], 'image': (0, 0, 0), 'weight': 3.4778161424304006, 'site_index': 6, 'wyckoff_sequence': 4, 'local_in

# wrap the distance function etc to a class

In [2]:
import numpy as np

import json
import logging
from kmcpy.external.pymatgen_structure import Structure
from kmcpy.external.pymatgen_local_env import CutOffDictNN
import itertools


class cluster_matcher():
    
    def __init__(self,neighbor_species=(('Cl-', 4),('Li+', 8)),reference_distance_matrix=np.array([[0,1],[1,0]]),reference_neighbor_sequence=[{}],neighbor_species_respective_reference_distance_matrix_dict={"Cl-":np.array([[0,1],[1,0]]),"Li+":np.array([[0,1],[1,0]])},neighbor_species_respective_reference_neighbor_sequence_dict={"Cl-":[{}],"Li+":[{}]}):
        
        self.neighbor_species=neighbor_species
        self.reference_distance_matrix=reference_distance_matrix
        self.neighbor_species_respective_reference_distance_matrix_dict=neighbor_species_respective_reference_distance_matrix_dict
        self.neighbor_species_respective_reference_neighbor_sequence_dict=neighbor_species_respective_reference_neighbor_sequence_dict
        self.reference_neighbor_sequence=reference_neighbor_sequence
        
        
        logging.info("a cluster is created with "+str(self.neighbor_species))
        logging.info("the distance matrix is "+str(self.reference_distance_matrix))
        
        
        pass
    
    @classmethod
    def from_reference_neighbor_sequences(self,reference_neighbor_sequences=[{}]):
        cn_dict={}
        neighbor_species_respective_reference_neighbor_sequence_dict={}
        
        for neighbor in reference_neighbor_sequences:
            
            site_element = neighbor["site"].species_string
            
            if site_element not in cn_dict:
                cn_dict[site_element] = 1
            else:
                cn_dict[site_element] += 1
            if site_element not in neighbor_species_respective_reference_neighbor_sequence_dict:
                neighbor_species_respective_reference_neighbor_sequence_dict[site_element]=[neighbor]
            else:
                neighbor_species_respective_reference_neighbor_sequence_dict[site_element].append(neighbor)
        
            
        neighbor_species_respective_reference_distance_matrix_dict={}
        
        for species in neighbor_species_respective_reference_neighbor_sequence_dict:
            neighbor_species_respective_reference_distance_matrix_dict[species]=self.build_distance_matrix_from_getnninfo_output(neighbor_species_respective_reference_neighbor_sequence_dict[species])
        
        neighbor_species=tuple(sorted(cn_dict.items(),key=lambda x:x[0]))
        
        reference_distance_matrix=self.build_distance_matrix_from_getnninfo_output(reference_neighbor_sequences)
        
                
        return cluster_matcher(neighbor_species=neighbor_species,reference_distance_matrix=reference_distance_matrix,reference_neighbor_sequence=reference_neighbor_sequences,neighbor_species_respective_reference_distance_matrix_dict=neighbor_species_respective_reference_distance_matrix_dict,neighbor_species_respective_reference_neighbor_sequence_dict=neighbor_species_respective_reference_neighbor_sequence_dict)
        
    
    @classmethod
    def build_distance_matrix_from_getnninfo_output(self,cutoffdnn_output=[{}]):
        """build a distance matrix from the output of CutOffDictNN.get_nn_info

        nn_info looks like: 
        [{'site': PeriodicSite: Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0), 'weight': 3.7390091507903174, 'site_index': 39, 'wyckoff_sequence': 15, 'local_index': 123, 'label': 'Si1'}, {'site': PeriodicSite: Na+ (-1.2831, -2.6519, 9.2421) [-0.3063, -0.3333, 0.4167], 'image': (-1, -1, 0), 'weight': 3.4778161424304046, 'site_index': 23, 'wyckoff_sequence': 17, 'local_index': 35, 'label': 'Na2'}, {'site': ...]
        
        or say:
        
        nn_info is a list, the elements of list is dictionary, the keys of dictionary are: "site":pymatgen.site, "wyckoff_sequence": ....
        
        Use the site.distance function to build matrix
        

        Args:
            cutoffdnn_output (nn_info, optional): nninfo. Defaults to reference_neighbor_sequences.

        Returns:
            np.2darray: 2d distance matrix, in format of numpy.array. The Column and the Rows are following the input sequence.
        """
    
        distance_matrix=np.zeros(shape=(len(cutoffdnn_output),len(cutoffdnn_output)))
          

        for sitedictindex1 in range(0,len(cutoffdnn_output)):
            for sitedictindex2 in range(0,sitedictindex1):
                """Reason for jimage=[0,0,0]
                
                site.distance is calculated by frac_coord1-frac_coord0 and get the cartesian distance. Note that for the two sites in neighbors,  the frac_coord itself already contains the information of jimage. For exaple:Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0),  see that the frac_coord of this Si4+ is not normalized to (0,1)!

                .
                """
                distance_matrix[sitedictindex1][sitedictindex2]=cutoffdnn_output[sitedictindex1]["site"].distance(cutoffdnn_output[sitedictindex2]["site"],jimage=[0,0,0])
            
            
        
        return distance_matrix

    def brutal_match(self,unsorted_nninfo=[{}],rtol=0.01):
        
        unsorted_cluster=cluster_matcher.from_reference_neighbor_sequences(unsorted_nninfo)
        
        if self.neighbor_species!=unsorted_cluster.neighbor_species:
            raise ValueError("input cluster has different environment")
        
        sorted_neighbor_sequence_dict={}

        for specie in unsorted_cluster.neighbor_species_respective_reference_neighbor_sequence_dict:
            
            sorted_neighbor_sequence_dict[specie]=[]
            
            for possible_local_sequence in itertools.permutations(unsorted_cluster.neighbor_species_respective_reference_neighbor_sequence_dict[specie]):
                
                if np.allclose(self.build_distance_matrix_from_getnninfo_output(possible_local_sequence),self.neighbor_species_respective_reference_distance_matrix_dict[specie],rtol=rtol):
                    
                    sorted_neighbor_sequence_dict[specie].append(list(possible_local_sequence))
        
        #logging.warning(str(sorted_neighbor_sequence_dict))
        
        
        sorted_neighbor_sequence_list=[]
        
        for specie in sorted_neighbor_sequence_dict:
            sorted_neighbor_sequence_list.append(sorted_neighbor_sequence_dict[specie])
        
        for possible_complete_sequence in itertools.product(*sorted_neighbor_sequence_list):
            
            #logging.warning(str(possible_complete_sequence))
            
            re_sorted_neighbors_list=[]
            
            for neighbor in possible_complete_sequence:

                re_sorted_neighbors_list.extend(list(neighbor))               
        
            if np.allclose(self.build_distance_matrix_from_getnninfo_output(re_sorted_neighbors_list),self.reference_distance_matrix,rtol=rtol):
                return re_sorted_neighbors_list
  
        raise ValueError("sequence not founded!")                
        



nasicon=Structure.from_cif("EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif",primitive=True)

center_Na1=[0,1]
local_env_finder = CutOffDictNN({('Na+','Na+'):4,('Na+','Si4+'):4})


reference_neighbor_sequences=sorted(sorted(local_env_finder.get_nn_info(nasicon,center_Na1[0]),key=lambda x:x["wyckoff_sequence"]),key = lambda x:x["label"])     


a=cluster_matcher.from_reference_neighbor_sequences(reference_neighbor_sequences=reference_neighbor_sequences)
np.set_printoptions(precision=2)

print(a.neighbor_species)
print(a.reference_distance_matrix)
print(a.neighbor_species_respective_reference_distance_matrix_dict)
print(a.neighbor_species_respective_reference_neighbor_sequence_dict)


wrong_sequence_neighbor=sorted(sorted(local_env_finder.get_nn_info(nasicon,center_Na1[1]),key=lambda x:x["wyckoff_sequence"]),key = lambda x:x["label"]) 


a.brutal_match(wrong_sequence_neighbor,rtol=0.01)


(('Na+', 6), ('Si4+', 6))
[[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [5.1  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [5.1  5.1  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [4.73 6.96 4.73 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [4.73 4.73 6.96 5.1  0.   0.   0.   0.   0.   0.   0.   0.  ]
 [6.96 4.73 4.73 5.1  5.1  0.   0.   0.   0.   0.   0.   0.  ]
 [3.16 6.2  3.06 3.71 6.54 6.49 0.   0.   0.   0.   0.   0.  ]
 [3.06 3.16 6.2  6.49 3.71 6.54 5.63 0.   0.   0.   0.   0.  ]
 [6.2  3.06 3.16 6.54 6.49 3.71 5.63 5.63 0.   0.   0.   0.  ]
 [6.54 6.49 3.71 3.16 6.2  3.06 4.92 7.48 4.92 0.   0.   0.  ]
 [3.71 6.54 6.49 3.06 3.16 6.2  4.92 4.92 7.48 5.63 0.   0.  ]
 [6.49 3.71 6.54 6.2  3.06 3.16 7.48 4.92 4.92 5.63 5.63 0.  ]]
{'Na+': array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [5.1 , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [5.1 , 5.1 , 0.  , 0.  , 0.  , 0.  ],
       [4.73, 6.96, 4.73, 0.  , 0.  , 0.  ],
       [4.73, 4.73, 6.96, 5.1

[{'site': PeriodicSite: Na+ (1.2831, -2.6519, -1.8484) [-0.1103, -0.3897, 0.2500],
  'image': (-1, -1, 0),
  'weight': 3.477816142430401,
  'site_index': 2,
  'wyckoff_sequence': 0,
  'local_index': 18,
  'label': 'Na2'},
 {'site': PeriodicSite: Na+ (1.6550, 2.4371, -1.8484) [0.2500, -0.1103, -0.3897],
  'image': (0, -1, -1),
  'weight': 3.4778161424303997,
  'site_index': 4,
  'wyckoff_sequence': 2,
  'local_index': 20,
  'label': 'Na2'},
 {'site': PeriodicSite: Na+ (-2.9381, 0.2148, -1.8484) [-0.3897, 0.2500, -0.1103],
  'image': (-1, 0, -1),
  'weight': 3.4778161424304,
  'site_index': 3,
  'wyckoff_sequence': 1,
  'local_index': 19,
  'label': 'Na2'},
 {'site': PeriodicSite: Na+ (-1.6550, -2.4371, 1.8484) [-0.2500, 0.1103, 0.3897],
  'image': (-1, 0, 0),
  'weight': 3.477816142430402,
  'site_index': 6,
  'wyckoff_sequence': 4,
  'local_index': 22,
  'label': 'Na2'},
 {'site': PeriodicSite: Na+ (2.9381, -0.2148, 1.8484) [0.3897, -0.2500, 0.1103],
  'image': (0, -1, 0),
  'weight': 

# generate events version 3

In [ ]:
import numpy as np
def extract_clusters_from_primitive_cell(prim_cif_name="210.cif",local_env_cutoff_dict={("Li+","Cl-"):4.0,("Li+","Li+"):3.0},atom_identifier_type="specie",center_atom_identifier="Li+",diffuse_to_atom_identifier="Li+",species_to_be_removed=["O2-","O"],force_distance_matrix_be_same=True,convert_to_primitive_cell=False,create_reference_cluster=True,verbosity=0):
    import json
    import logging
    from kmcpy.external.pymatgen_structure import Structure
    from kmcpy.external.pymatgen_local_env import CutOffDictNN
    
    # --------------
    
    
    logging.basicConfig(level=verbosity,handlers=[
        logging.FileHandler("debug.log"),
        logging.StreamHandler()
    ])
    logging.warning("Extracting clusters from primitive cell structure. This primitive cell can also be grain boundary")
    
    primitive_cell=Structure.from_cif(prim_cif_name,primitive=convert_to_primitive_cell)
    primitive_cell.add_oxidation_state_by_guess()
    primitive_cell.remove_species(species_to_be_removed)
    
    logging.warning("primitive cell composition after adding oxidation state and removing uninvolved species: ")
    
    logging.info(primitive_cell.composition)
    
    logging.warning("building center atom index list")
    
    
    center_atom_indices=[]    
    if atom_identifier_type=="specie":
        for i in range(0,len(primitive_cell)):
            if center_atom_identifier in primitive_cell[i].species:
                center_atom_indices.append(i)
                
    elif atom_identifier_type=="label":

        for i in range(0,len(primitive_cell)):
            if primitive_cell[i].properties["label"]==center_atom_identifier:
                center_atom_indices.append(i)
                
    elif atom_identifier_type=="list":
        center_atom_indices=center_atom_identifier
    
    else:
        raise ValueError('unrecognized atom_identifier_type. Please select from: ["specie","label","list"] ')
    
    logging.info("please check if these are center atom:")
    for i in center_atom_indices:
        
        logging.info(str(primitive_cell[i]))        
        
        
        
    #--------
    
    local_env_finder = CutOffDictNN(local_env_cutoff_dict)
    
    reference_distance_matrix_dict={}
    
    local_env_info_dict = {}
   
   
    def build_distance_matrix_from_getnninfo_output(cutoffdnn_output=[{}]):
        """build a distance matrix from the output of CutOffDictNN.get_nn_info

        nn_info looks like: 
        [{'site': PeriodicSite: Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0), 'weight': 3.7390091507903174, 'site_index': 39, 'wyckoff_sequence': 15, 'local_index': 123, 'label': 'Si1'}, {'site': PeriodicSite: Na+ (-1.2831, -2.6519, 9.2421) [-0.3063, -0.3333, 0.4167], 'image': (-1, -1, 0), 'weight': 3.4778161424304046, 'site_index': 23, 'wyckoff_sequence': 17, 'local_index': 35, 'label': 'Na2'}, {'site': ...]
        
        or say:
        
        nn_info is a list, the elements of list is dictionary, the keys of dictionary are: "site":pymatgen.site, "wyckoff_sequence": ....
        
        Use the site.distance function to build matrix
        

        Args:
            cutoffdnn_output (nn_info, optional): nninfo. Defaults to reference_neighbor_sequences.

        Returns:
            np.2darray: 2d distance matrix, in format of numpy.array. The Column and the Rows are following the input sequence.
        """
    
        distance_matrix=np.zeros(shape=(len(cutoffdnn_output),len(cutoffdnn_output)))
          

        for sitedictindex1 in range(0,len(cutoffdnn_output)):
            for sitedictindex2 in range(0,len(cutoffdnn_output)):
                """Reason for jimage=[0,0,0]
                
                site.distance is calculated by frac_coord1-frac_coord0 and get the cartesian distance. Note that for the two sites in neighbors,  the frac_coord itself already contains the information of jimage. For exaple:Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0),  see that the frac_coord of this Si4+ is not normalized to (0,1)!

                .
                """
                distance_matrix[sitedictindex1][sitedictindex2]=cutoffdnn_output[sitedictindex1]["site"].distance(cutoffdnn_output[sitedictindex2]["site"],jimage=[0,0,0])
            
            
        
        return distance_matrix
    
    def nn_info_to_element_number_tuple(cutoffdnn_output=[{}]):
        """convert the nn_info into cn_info,key=tuple(tuple(str(specie),int(number in neighbors)))

        Args:
            cutoffdnn_output (list, optional): list of dictionary, which is the get_nn_info output. Defaults to [{}].

        Returns:
            tuple(tuple(str(specie),int(number in neighbors))): tuple as the key
            example:
            (  ('Cl-', 4),   ('Li+', 8)   )
        """
        cn_dict={}
        for neighbor in cutoffdnn_output:
            
            site_element = neighbor["site"].species_string
            if site_element not in cn_dict:
                cn_dict[site_element] = 1
            else:
                cn_dict[site_element] += 1
                
        return tuple(sorted(cn_dict.items(),key=lambda x:x[0]))
    
    def brutely_rearrange_neighbors(wrong_neighbor_sequence=[],corect_neighbor_sequence=[],reference_distance_matrix=[],verbose=False,rtol_for_neighbor=0.01):
        """
        reference distance matrix: numpy.2Darray
        wrong neighbor sequence: list of dictionary
        
        [{'site': PeriodicSite: Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0), 'weight': 3.7390091507903174, 'site_index': 39, 'wyckoff_sequence': 15, 'local_index': 123, 'label': 'Si1'}, {'site': PeriodicSite: Na+ (-1.2831, -2.6519, 9.2421) [-0.3063, -0.3333, 0.4167], 'image': (-1, -1, 0), 'weight': 3.4778161424304046, 'site_index': 23, 'wyckoff_sequence': 17, 'local_index': 35, 'label': 'Na2'}, {'site': ...}]
        
        for nasicon, wrong neighbor sequence contains 6 neighbors of Si and 6 neighbors of Na
        """
        
        complete_list_for_iteration=[]
        
        dict_by_label={}
        
        for neighbor_information in wrong_neighbor_sequence:# neighbor information: dictionary: {'site': PeriodicSite: Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0), 'weight': 3.7390091507903174, 'site_index': 39, 'wyckoff_sequence': 15, 'local_index': 123, 'label': 'Si1'}
            # sort the input neighbor sequence by label
            if neighbor_information["label"] not in dict_by_label:
                dict_by_label[neighbor_information["label"]]=[neighbor_information]
            else:
                dict_by_label[neighbor_information["label"]].extend([neighbor_information])

                
        """
        dict_by_label={"Na2":[{"site":xxx,"index":xxx,...},{"site":xxx,"index":xxx,...},...],
        
                        "Si1:[]...
        }
        
        dict_by_label["label"]=list
        dict_by_label["label"][0]=dic, with key=site,image, wyckoff_sequence,...
        """
        
        for label in dict_by_label:
            """
            complete_list_for_iteration=List[itertools.permutation()]
            every element in the list are permutations of neighbors with 1 kind of label
            
            for nasicon
            
            complete_list_for_iteration=[itertools.permutation($"sites with label Na2"),itertools.permutation($"sites with label Si2")]
            
            for example , if there is 2 Na2 and 3 Si2 
            
            complete_list_for_iteration=[(Na2_1,Na2_2 or Na2_2, Na2_1), (Si1Si2Si3,Si1Si3Si2,Si2Si1Si3,Si2Si3Si1,Si3Si2Si1,Si3Si1Si2)]
            """
            complete_list_for_iteration.append(itertools.permutations(dict_by_label[label]))
        


        print_divider()
        print(" trying to search for a correct neighbor sequence.  progress is stored in logfile. As long as the array is changing, it should be working properly. It take around 90s for searching 12 neighbors. For more neighbors, the searching time increase in O(n!). ")        
        for re_sorted_neighbors_tuple in (itertools.product(*complete_list_for_iteration)):

            """re_sorted_neighbors_tuple is tuple format from itertools.product
            
            need to convert it to list 
            
            it looks like this :
            
            re_sorted_neighbors_tuple=(  (Na2-1,Na2-3,Na2-2) , (Si1-5,Si1-2) ... )

            convert them to plain list
            
            the itertools.product is the product of the permutations 
    
            for example , if there is 2 Na2 and 3 Si2 
            
            complete_list_for_iteration=[(Na2_1,Na2_2 or Na2_2, Na2_1), (Si1Si2Si3,Si1Si3Si2,Si2Si1Si3,Si2Si3Si1,Si3Si2Si1,Si3Si1Si2)]
                        
            for Na2_1 Na2_2 this sequence, try all permutations of Si  (Si1Si2Si3,Si1Si3Si2,Si2Si1Si3,Si2Si3Si1,Si3Si2Si1,Si3Si1Si2)
            then for Na2_2 Na2_1 this sequence ,try all permutations of Si again (Si1Si2Si3,Si1Si3Si2,Si2Si1Si3,Si2Si3Si1,Si3Si2Si1,Si3Si1Si2)
            then if there is any more Na2 permutation , try all Si permutation
            ....
            
            """
            re_sorted_neighbors_list=[]
        
            if verbose:
                with open("searching.log","a+") as f:
                    # this is the progress for impatient people
                    for neighbor in re_sorted_neighbors_tuple:
                        re_sorted_neighbors_list.extend(list(neighbor))
                    sequences=[]
                    for neighbor in re_sorted_neighbors_list:
                        sequences.append(str(neighbor["wyckoff_sequence"]))
                    f.write(",".join(sequences))
                    
            else:
                for neighbor in re_sorted_neighbors_tuple:

                    re_sorted_neighbors_list.extend(list(neighbor))                

            if np.allclose(build_distance_matrix_from_getnninfo_output(cutoffdnn_output=re_sorted_neighbors_list),reference_distance_matrix,rtol=rtol_for_neighbor):
                print_divider()
                print("Reorganized neighbors found. In this sequence of neighbor, the distance matrix should be same to the reference matrix ")
                print([(neighbor["wyckoff_sequence"],neighbor["label"]) for neighbor in re_sorted_neighbors_list])
                print("distance matrix is ")
                print(build_distance_matrix_from_getnninfo_output(cutoffdnn_output=re_sorted_neighbors_list))
                print(" use this as the new sequence of neighbor.")
                return re_sorted_neighbors_list
        raise ValueError("Unfortunately, generate_events2.brutely_rearrange_neighbors() cannot find any way to rearrange the local environment to get a same distance matrix! Probably the tolerance of np.allclose is too small? This shouldn't happen. Please also check if you are working on a correct local cluster....")

    
    
    reference_cluster_type=0
    
    for center_atom_index in center_atom_indices:
        
        unsorted_neighbor_sequences=sorted(sorted(local_env_finder.get_nn_info(primitive_cell,center_atom_index),key=lambda x:x["wyckoff_sequence"]),key = lambda x:x["label"])      
                    
        this_center_atom_cn_tuples=nn_info_to_element_number_tuple(unsorted_neighbor_sequences)
        
        if this_center_atom_cn_tuples not in reference_distance_matrix_dict:
            
            if create_reference_cluster:
                reference_cluster_sites=[primitive_cell[center_atom_index]]
                
                for i in unsorted_neighbor_sequences:
                    
                    reference_cluster_sites.append(i["site"])
                    reference_cluster_structure=Structure.from_sites(sites=reference_cluster_sites)
                    reference_cluster_structure.to("cif",str(reference_cluster_type)+"th_reference_cluster.cif") 
            
            reference_distance_matrix_dict[this_center_atom_cn_tuples]=build_distance_matrix_from_getnninfo_output(unsorted_neighbor_sequences)
            

            
        if force_distance_matrix_be_same:
            pass
                
            
            
    logging.info(str(reference_distance_matrix_dict))
    
        
    
    
    
    raise NotImplementedError
    primitive_cell.remove_species(species_to_be_removed)
    
    
    
    
    
    
    

    
    
extract_clusters_from_primitive_cell(prim_cif_name="210.cif",local_env_cutoff_dict={("Li+","Cl-"):4.0,("Li+","Li+"):3.0},atom_identifier_type="specie",center_atom_identifier="Li+",diffuse_to_atom_identifier="Li+",species_to_be_removed=["O2-","O"],force_distance_matrix_be_same=True,convert_to_primitive_cell=False,create_reference_cluster=True,verbosity=0)
    

test on new method of rearranging neighbor

In [36]:
a=[[1],[2]]
def showww(a,b):
    print(a,b)
showww(*a)

[1] [2]
